In [ ]:
import torch
import sys
sys.path.append('../atari')
# Install the missing 'models' module

weight_dir = '/lichenghao/lzh/componet/experiments/atari/agents/Freeway/FuseEncoderActor/'
weight_paths = [ weight_dir + f"Freeway_{i}_FuseNet_42" for i in range(8)]
a = torch.load(weight_paths[0]+"/actor.pt") # fuck, 轻易不要改模型结构
# for i in weight_paths:
#     a = torch.load(i+"/actor.pt")

AttributeError: Can't get attribute 'FuseLinear' on <module 'models.fusenet' from '/lichenghao/lzh/componet/experiments/data_analysis/../atari/models/fusenet.py'>